#Instalamos Librerias para cargar el modelo de OpenAI

In [ ]:
!pip install openai

#Configuramos el tokken de permiso para acceso al modelo

In [ ]:
import openai
import os

apikey_openai = ''
os.environ['OPENAI_API_KEY'] = apikey_openai

openai.api_key = apikey_openai

#Carga de modelo semantico

In [ ]:
from sentence_transformers import SentenceTransformer, util

# Modelo para similitud semántica (opcional)
semantic_model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#Funciones de Evaluación

In [ ]:
def test_openai_model(model_name, prompt, max_tokens=500):
    response = openai.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system",
             "content": "Eres un asistente experto en análisis financiero."},
            {"role": "user",
             "content": prompt},
        ],
        max_tokens=max_tokens,
        temperature=0.7,
    )
    return response.choices[0].message.content.strip()

In [ ]:
def evaluate_coherence(model_output, reference=None):
    if reference:
        # Similitud semántica con referencia (si hay un texto base)
        embedding1 = semantic_model.encode(model_output, convert_to_tensor=True)
        embedding2 = semantic_model.encode(reference, convert_to_tensor=True)
        similarity = util.pytorch_cos_sim(embedding1, embedding2).item()
        return similarity  # Retorna un valor entre 0 y 1
    else:
        # Evaluación manual (si no hay referencia)
        print("Evaluar manualmente la coherencia (1-5):")
        print("Texto generado:", model_output)
        score = input("Puntaje (1-5): ")
        return float(score)

In [ ]:
def evaluate_long_context(model_name, prompt, context):
    long_prompt = f"{context}\nPregunta: {prompt}"
    response = test_openai_model(model_name, long_prompt)
    print("Respuesta del modelo:", response)

    # Evaluar precisión manualmente
    print("Evaluar precisión (1-5):")
    score = input("Puntaje (1-5): ")
    return float(score)

In [ ]:
import time

def evaluate_speed(model_name, prompt):
    start_time = time.time()
    response = test_openai_model(model_name, prompt)
    end_time = time.time()
    duration = end_time - start_time
    print(f"Tiempo de generación: {duration:.2f} segundos")
    return duration

In [ ]:
def calculate_cost(model_name, prompt, response):
    # Calcular tokens usados (entrada + salida)
    prompt_tokens = len(prompt.split())  # Simplificación: 1 token ≈ 1 palabra
    response_tokens = len(response.split())
    total_tokens = prompt_tokens + response_tokens

    # Tarifas según el modelo
    pricing = {
        "gpt-3.5-turbo": (0.0015, 0.002),
        "gpt-3.5-turbo-16k": (0.003, 0.004),
        "gpt-4": (0.03, 0.06),
        "gpt-4-turbo": (0.03, 0.06)
    }
    input_price, output_price = pricing[model_name]

    # Costo estimado
    cost = (prompt_tokens * input_price + response_tokens * output_price) / 1000
    print(f"Costo estimado: ${cost:.4f} USD")
    return cost

In [ ]:
# Configurar prompt y contexto
prompt = "Explícame cómo los modelos de lenguaje pueden mejorar el análisis financiero."

referencia = "Los modelos de lenguaje largo, como GPT-4, mejoran significativamente el análisis financiero al procesar grandes \
              volúmenes de datos textuales, como informes económicos, noticias y documentos regulatorios. Estas capacidades \
              permiten extraer patrones, identificar riesgos y oportunidades, y generar reportes automáticos personalizados \
              para tomar decisiones informadas. Además, su capacidad para mantener un contexto amplio facilita la comprensión \
              de relaciones complejas en los datos, lo cual es crucial para realizar predicciones precisas y análisis detallados."

prompt_largo = """
En el ámbito financiero, el análisis de sentimiento se refiere al uso de técnicas de procesamiento de lenguaje natural para identificar y extraer información subjetiva de textos como noticias, publicaciones en redes sociales, reportes económicos y más. Este análisis es fundamental para predecir movimientos en los mercados bursátiles, ya que las emociones de los inversores pueden influir significativamente en las tendencias del mercado. Por ejemplo, un análisis de noticias negativas sobre una compañía puede llevar a una venta masiva de sus acciones, mientras que noticias positivas pueden incrementar la demanda.

El análisis de sentimiento no solo se limita a identificar emociones positivas o negativas, sino que también permite clasificar opiniones neutrales y medir su intensidad. Los modelos de lenguaje de contexto largo, como GPT-4, tienen la capacidad de procesar grandes cantidades de datos textuales, identificar patrones, y correlacionar eventos noticiosos con movimientos en los precios de las acciones. Esto resulta especialmente útil en mercados altamente volátiles, donde el tiempo de reacción es crítico.

En un estudio reciente, se analizaron más de 10,000 publicaciones relacionadas con el sector tecnológico en un período de tres meses. Los resultados mostraron que el análisis de sentimiento basado en modelos de lenguaje puede predecir movimientos de precios con una precisión del 75%, lo que supera significativamente las técnicas tradicionales basadas únicamente en datos cuantitativos. Sin embargo, estos modelos enfrentan desafíos, como la interpretación de sarcasmo o la identificación de información irrelevante, que pueden sesgar los resultados. Por tanto, una integración adecuada de análisis cuantitativo y cualitativo es esencial para optimizar los resultados."*

Preguntas:

Según el texto, ¿por qué es importante el análisis de sentimiento en el ámbito financiero? responde en máximo 10 palabras.
"""

context = "Los modelos de lenguaje largo permiten manejar grandes volúmenes de información..."

#Evaluación de cada Modelo

##Coherencia

###GPT-4 Turbo

In [ ]:
print("GPT-4 Turbo:")
model_name = "gpt-4-turbo"
response = test_openai_model(model_name, prompt)

coherence_score = evaluate_coherence(response)

print(f"Coherencia de Evaluación según Experto: {coherence_score}")

GPT-4 Turbo:
Evaluar manualmente la coherencia (1-5):
Texto generado: Los modelos de lenguaje, como los desarrollos recientes en inteligencia artificial y aprendizaje automático, pueden desempeñar un papel crucial en mejorar el análisis financiero de varias maneras. Aquí hay algunas de las principales áreas en las que estos modelos pueden contribuir significativamente:

### 1. **Análisis de Sentimiento**
Los modelos de lenguaje pueden procesar grandes volúmenes de texto, como noticias, informes financieros, redes sociales y discusiones en foros, para determinar el sentimiento general hacia una acción, sector o mercado. Por ejemplo, un aumento en el sentimiento positivo en las noticias y las redes sociales sobre una compañía podría anticipar un aumento en su precio de acción.

### 2. **Automatización de Informes**
Los modelos de lenguaje pueden ayudar a automatizar la creación de informes financieros, generando resúmenes de estados financieros, reportes de ganancias y otros documentos r

In [ ]:
print("GPT-4 Turbo:")
model_name = "gpt-4-turbo"
response = test_openai_model(model_name, prompt)

coherence_score = evaluate_coherence(response, reference=referencia)

if(coherence_score*5 <= 1):
  score = 1
else:
  score = coherence_score*5

print("Puntaje (1-5):")
print(f"Coherencia de Similitud Semántica : {score}")

GPT-4 Turbo:
Puntaje (1-5):
Coherencia de Similitud Semántica : 3.5854339599609375


###GPT-4

In [ ]:
print("GPT-4:")
model_name = "gpt-4"

response = test_openai_model(model_name, prompt)

coherence_score = evaluate_coherence(response)

print(f"Coherencia de Evaluación según Experto: {coherence_score}")

GPT-4:
Evaluar manualmente la coherencia (1-5):
Texto generado: Los modelos de lenguaje pueden mejorar el análisis financiero de varias maneras:

1. Análisis de Sentimientos: Los modelos de lenguaje pueden analizar el sentimiento de los informes financieros, las noticias y las redes sociales para determinar la percepción del mercado sobre una empresa o activo específico. Esta información puede ser utilizada para predecir tendencias de mercado y tomar decisiones de inversión más informadas.

2. Resumen Automático: Los modelos de lenguaje pueden resumir automáticamente informes financieros largos y complicados, lo que permite a los analistas acceder a la información clave más rápidamente.

3. Detección de Fraudes: Los modelos de lenguaje pueden analizar patrones de lenguaje en las comunicaciones financieras para detectar posibles fraudes. Por ejemplo, pueden identificar cambios en el lenguaje que sugieran que una empresa está ocultando información financiera negativa.

4. Análisis Predic

In [ ]:
print("GPT-4:")
model_name = "gpt-4"
response = test_openai_model(model_name, prompt)

coherence_score = evaluate_coherence(response, reference=referencia)

if(coherence_score*5 <= 1):
  score = 1
else:
  score = coherence_score*5

print("Puntaje (1-5):")
print(f"Coherencia de Similitud Semántica : {score}")

GPT-4:
Puntaje (1-5):
Coherencia de Similitud Semántica : 3.3827120065689087


###GPT-3.5 Turbo

In [ ]:
print("GPT-3.5 Turbo:")
model_name = "gpt-3.5-turbo"

response = test_openai_model(model_name, prompt)

coherence_score = evaluate_coherence(response)

print(f"Coherencia de Evaluación según Experto: {coherence_score}")

GPT-3.5 Turbo:
Evaluar manualmente la coherencia (1-5):
Texto generado: Los modelos de lenguaje son herramientas de inteligencia artificial que pueden mejorar el análisis financiero de diversas maneras:

1. Procesamiento de texto: Los modelos de lenguaje pueden analizar grandes cantidades de texto, como informes financieros, noticias, redes sociales y otros documentos, para extraer información relevante para la toma de decisiones financieras.

2. Sentimiento del mercado: Los modelos de lenguaje pueden analizar el sentimiento del mercado a partir de noticias y comentarios en línea, lo que puede ayudar a los inversores a comprender mejor el estado de ánimo de los inversores y anticipar posibles movimientos del mercado.

3. Análisis de riesgos: Los modelos de lenguaje pueden ayudar a identificar y evaluar diferentes riesgos financieros al analizar información no estructurada, como reportes de noticias, discursos de ejecutivos, informes regulatorios, entre otros.

4. Automatización de tare

In [ ]:
print("GPT-3.5 Turbo:")
model_name = "gpt-3.5-turbo"
response = test_openai_model(model_name, prompt)

coherence_score = evaluate_coherence(response, reference=referencia)

if(coherence_score*5 <= 1):
  score = 1
else:
  score = coherence_score*5

print("Puntaje (1-5):")
print(f"Coherencia de Similitud Semántica : {score}")

GPT-3.5 Turbo:
Puntaje (1-5):
Coherencia de Similitud Semántica : 3.9457207918167114


###GPT-3.5 Turbo-16k

In [ ]:
print("GPT-3.5 Turbo-16k:")
model_name = "gpt-3.5-turbo-16k"
response = test_openai_model(model_name, prompt)

coherence_score = evaluate_coherence(response)

print(f"Coherencia de Evaluación según Experto: {coherence_score}")

GPT-3.5 Turbo-16k:
Evaluar manualmente la coherencia (1-5):
Texto generado: Los modelos de lenguaje, como el procesamiento del lenguaje natural (NLP, por sus siglas en inglés) y la inteligencia artificial (IA), pueden mejorar el análisis financiero de varias maneras. Aquí te presento algunas de ellas:

1. Análisis de sentimiento: Los modelos de lenguaje pueden ayudar a analizar el sentimiento de los textos financieros, como noticias, informes de empresas o redes sociales. Esto permite identificar si la información tiene una connotación positiva o negativa, lo que puede ayudar a tomar decisiones de inversión informadas.

2. Extracción de información: Los modelos de lenguaje pueden extraer información relevante de grandes cantidades de datos financieros no estructurados. Por ejemplo, pueden identificar nombres de empresas, fechas, cifras financieras, tendencias y eventos importantes en informes anuales o conferencias de inversores.

3. Predicción de precios: Los modelos de lenguaje puede

In [ ]:
print("GPT-3.5 Turbo-16k:")
model_name = "gpt-3.5-turbo-16k"

response = test_openai_model(model_name, prompt)

coherence_score = evaluate_coherence(response, reference=referencia)

if(coherence_score*5 <= 1):
  score = 1
else:
  score = coherence_score*5

print("Puntaje (1-5):")
print(f"Coherencia de Similitud Semántica : {score}")

GPT-3.5 Turbo-16k:
Puntaje (1-5):
Coherencia de Similitud Semántica : 3.878626525402069


##Capacidad de contexto largo

In [ ]:
print("GPT-4 Turbo:")
model_name = "gpt-4-turbo"

context_score = evaluate_long_context(model_name, prompt_largo, context)

print(f"Contexto Largo: {context_score}")

GPT-4 Turbo:
Respuesta del modelo: Para predecir movimientos de mercado influenciados por emociones de inversores.
Evaluar precisión (1-5):
Puntaje (1-5): 5
Contexto Largo: 5.0


In [ ]:
print("GPT-4:")
model_name = "gpt-4"

context_score = evaluate_long_context(model_name, prompt_largo, context)

print(f"Contexto Largo: {context_score}")

GPT-4:
Respuesta del modelo: Predice movimientos de mercado basados en emociones de inversores.
Evaluar precisión (1-5):
Puntaje (1-5): 5
Contexto Largo: 5.0


In [ ]:
print("GPT-3.5 Turbo:")
model_name = "gpt-3.5-turbo"

context_score = evaluate_long_context(model_name, prompt_largo, context)

print(f"Contexto Largo: {context_score}")

GPT-3.5 Turbo:
Respuesta del modelo: Importante para predecir movimientos en mercados bursátiles.
Evaluar precisión (1-5):
Puntaje (1-5): 4.5
Contexto Largo: 4.5


In [ ]:
print("GPT-3.5 Turbo-16k:")
model_name = "gpt-3.5-turbo-16k"

context_score = evaluate_long_context(model_name, prompt_largo, context)

print(f"Contexto Largo: {context_score}")

GPT-3.5 Turbo-16k:
Respuesta del modelo: Es importante para predecir movimientos en los mercados bursátiles.
Evaluar precisión (1-5):
Puntaje (1-5): 4.8
Contexto Largo: 4.8


##Velocidad

In [ ]:
print("GPT-4 Turbo:")
model_name = "gpt-4-turbo"

speed = evaluate_speed(model_name, prompt)

print(f"Velocidad: {speed:.2f} s")

GPT-4 Turbo:
Tiempo de generación: 19.34 segundos
Velocidad: 19.34 s


In [ ]:
print("GPT-4:")
model_name = "gpt-4"

speed = evaluate_speed(model_name, prompt)

print(f"Velocidad: {speed:.2f} s")

GPT-4:
Tiempo de generación: 27.10 segundos
Velocidad: 27.10 s


In [ ]:
print("GPT-3.5 Turbo:")
model_name = "gpt-3.5-turbo"

speed = evaluate_speed(model_name, prompt)

print(f"Velocidad: {speed:.2f} s")

GPT-3.5 Turbo:
Tiempo de generación: 4.52 segundos
Velocidad: 4.52 s


In [ ]:
print("GPT-3.5 Turbo-16k:")
model_name = "gpt-3.5-turbo-16k"

speed = evaluate_speed(model_name, prompt)

print(f"Velocidad: {speed:.2f} s")

GPT-3.5 Turbo-16k:
Tiempo de generación: 4.21 segundos
Velocidad: 4.21 s


##Costo estimado

In [ ]:
print("GPT-4 Turbo:")
model_name = "gpt-4-turbo"

cost = calculate_cost(model_name, prompt, response)

print(f"Costo: ${cost:.4f}")

GPT-4 Turbo:
Costo estimado: $0.0196 USD
Costo: $0.0196


In [ ]:
print("GPT-4:")
model_name = "gpt-4"

cost = calculate_cost(model_name, prompt, response)

print(f"Costo: ${cost:.4f}")

GPT-4:
Costo estimado: $0.0196 USD
Costo: $0.0196


In [ ]:
print("GPT-3.5 Turbo:")
model_name = "gpt-3.5-turbo"

cost = calculate_cost(model_name, prompt, response)

print(f"Costo: ${cost:.4f}")

GPT-3.5 Turbo:
Costo estimado: $0.0007 USD
Costo: $0.0007


In [ ]:
print("GPT-3.5 Turbo-16k:")
model_name = "gpt-3.5-turbo-16k"

cost = calculate_cost(model_name, prompt, response)

print(f"Costo: ${cost:.4f}")

GPT-3.5 Turbo-16k:
Costo estimado: $0.0013 USD
Costo: $0.0013
